In [ ]:
!wget https://www.lamsade.dauphine.fr/~cazenave/project2022.zip
!unzip project2022.zip

In [3]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import layers, regularizers
import gc
import golois
import os
from google.colab import files
import os, pathlib, shutil, random
from tensorflow.keras.layers import MultiHeadAttention
import csv
import matplotlib.pyplot as plt
import pandas as pd

In [42]:
# List files in the current directory
files_in_directory = os.listdir()
print(files_in_directory)

['.config', 'Game.h', 'conv2.csv', 'golois.cpp', 'zip.sh', 'Rzone.h', 'compile.sh', 'importGolois.ipynb', 'golois.cpython-38-x86_64-linux-gnu.so', 'golois.py', 'golois.cpython-37m-x86_64-linux-gnu.so', 'Board.h', 'games.data', 'golois.cpython-310-x86_64-linux-gnu.so', 'project2022.zip', 'compileMAC.sh', 'HungNGUYEN_cnn.h5', 'golois.cpython-39-x86_64-linux-gnu.so', 'validation.data', 'sample_data']


In [40]:
#Upload pretrained model
uploaded = files.upload()

Saving HungNGUYEN_EFA.h5 to HungNGUYEN_EFA.h5


In [5]:
def file_remove(filename):
  file_to_remove = filename
  if os.path.exists(file_to_remove):
      os.remove(file_to_remove)
      print(f"{file_to_remove} has been removed.")
  else:
      print(f"{file_to_remove} does not exist.")

In [44]:
file_remove('HungNGUYEN_cnn.h5')

HungNGUYEN_cnn.h5 has been removed.


**Define Inputs and Parameters**

In [45]:
#Go parameters
planes = 31
moves = 361
N = 20000
epochs = 40
batch = 128
filters = 32
dropout_rate = 0.2

In [46]:
#Define input
input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype('float32')

policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical(policy)

value = np.random.randint(2, size=(N,))
value = value.astype('float32')

end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype('float32')

groups = np.zeros((N, 19, 19, 1))
groups = groups.astype('float32')

print("getValidation", flush=True)
golois.getValidation(input_data, policy, value, end)

getValidation


**CNN**

In [52]:
input = keras.Input(shape=(19, 19, planes), name='board')
x = layers.Conv2D(filters, 1, activation='relu', padding='same')(input)
for i in range (7):
    x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)

**ResNet**

In [ ]:
def resnet_block(ident, filters):
    ident = ident
    x = layers.Conv2D(filters, 3, activation='relu', padding='same')(ident)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters, 3, activation=None, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.add([ident, x])
    x = layers.Activation('relu')(x)
    return x

In [ ]:
#resnet
input = keras.Input(shape=(19, 19, planes), name='board')
x = layers.Conv2D(filters, 1, activation='relu', padding='same')(input)
for _ in range(4):
    x=resnet_block(x,filters)

**MobileNet**

In [ ]:
def bottleneck_block(x, expand=6*filters, squeeze=filters):

    m = layers.Conv2D(expand, (1,1), kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(x)
    m = layers.BatchNormalization()(m)
    m = layers.Activation('relu')(m)

    m = layers.DepthwiseConv2D((3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(m)
    m = layers.BatchNormalization()(m)
    m = layers.Activation('relu')(m)

    m = layers.Conv2D(squeeze, (1,1), kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(m)
    m = layers.BatchNormalization()(m)

    return layers.Add()([m, x])

In [ ]:
#MobileNet
input = keras.Input(shape=(19, 19, planes), name='board')
x = layers.Conv2D(filters, 1, activation='relu', padding='same')(input)
x = layers.Dropout(dropout_rate)(x)
for i in range (5):
    x = bottleneck_block(x)

**ConvNEXT ver 1**

In [37]:
input = keras.Input(shape=(19, 19, planes), name='board')
x = layers.Conv2D(filters, 1, activation='relu', padding='same')(input)
for i in range (7):
  x1 = tf.keras.layers.DepthwiseConv2D((7,7), padding='same', use_bias=False)(x)
  x1 = layers.LayerNormalization()(x1)
  x1 = layers.Conv2D(4 * filters, 1, padding='same', activation='gelu')(x1)
  x1 = layers.Dropout(dropout_rate)(x1)
  x1 = layers.Conv2D(filters, 1, padding='same')(x1)
  x = layers.add([x1, x])
x = tf.keras.layers.BatchNormalization()(x)



**ConvNEXT ver 2**

In [ ]:
#Both work, but the model below works better
'''
input = keras.Input(shape=(19, 19, planes), name='board')
x = layers.Conv2D(filters, 1, activation='relu', padding='same')(input)
x = layers.Dropout(dropout_rate)(x)
for i in range(7):
  x1 = layers.DepthwiseConv2D((7, 7), padding='same', use_bias=False)(x)
  x1 = layers.LayerNormalization()(x1)
  x1 = layers.Conv2D(4 * filters, 1, padding='same', activation='gelu')(x1)
  x1 = layers.BatchNormalization()(x1)
  x1 = layers.Conv2D(filters, 1, padding='same')(x1)
  x = layers.add([x1, x])
x = tf.keras.layers.BatchNormalization()(x)
'''

In [18]:
input = keras.Input(shape=(19, 19, planes), name='board')
x = layers.Conv2D(filters, 1, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.0001))(input)
x = layers.Dropout(dropout_rate)(x)
for i in range(7):
  x1 = layers.DepthwiseConv2D((7, 7), padding='same', use_bias=False, kernel_regularizer=regularizers.l2(0.0001))(x)
  x1 = layers.LayerNormalization()(x1)
  x1 = layers.Conv2D(4 * filters, 1, padding='same', activation='gelu', kernel_regularizer=regularizers.l2(0.0001))(x1)
  #x1 = layers.Dropout(dropout_rate)(x1)
  x1 = layers.BatchNormalization()(x1)
  x1 = layers.Conv2D(filters, 1, padding='same', kernel_regularizer=regularizers.l2(0.0001))(x1)
  x = layers.add([x1, x])
x = tf.keras.layers.BatchNormalization()(x)

**Transformers**

In [13]:
class MB4D(layers.Layer):
    def __init__(self, filters, dropout_rate=0.2, **kwargs):
        super(MB4D, self).__init__(**kwargs)
        self.filters = filters
        self.conv1x1_1 = layers.Conv2D(filters, 1, activation='relu', padding='same')
        self.batch_norm1 = layers.BatchNormalization()
        self.gelu = layers.Activation('gelu')
        self.conv1x1_2 = layers.Conv2D(filters, 1, activation='relu', padding='same')
        self.batch_norm2 = layers.BatchNormalization()

    def call(self, inputs, training=None):
        x1 = self.conv1x1_1(inputs)
        x1 = self.batch_norm1(x1, training=training)
        x1 = self.gelu(x1)
        x1 = self.conv1x1_2(x1)
        x1 = self.batch_norm2(x1, training=training)
        return x1

In [14]:
class MB3D(tf.keras.layers.Layer):
    def __init__(self, units, dropout_rate=0.1, **kwargs):
        super(MB3D, self).__init__(**kwargs)
        self.units = units
        self.dropout_rate = dropout_rate
        self.layer_norm = layers.LayerNormalization()
        self.linear = layers.Dense(units)
        self.linear_out = layers.Dense(units)
        self.gelu = layers.Activation('gelu')

    def call(self, inputs):
        x = self.layer_norm(inputs)
        linear_output = self.linear(x)
        key = linear_output
        query = linear_output
        value = linear_output

        attention_weights = tf.matmul(key, query, transpose_b=True) / tf.sqrt(tf.cast(self.units, dtype=tf.float32))
        attention_weights = tf.nn.softmax(attention_weights, axis=-1)
        attention_output = tf.matmul(attention_weights, value)
        attention_output_linear = self.linear_out(attention_output)

        result = layers.add([attention_output_linear, inputs])
        result = self.layer_norm(result)
        result = self.linear_out(result)
        result = self.gelu(result)
        result = self.linear_out(result)
        result = layers.add([result, value])

        return result


In [15]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=output_dim, trainable=False)
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[1]
        batch_size = tf.shape(inputs)[0]
        positions = tf.range(start=0, limit=length, delta=1)
        positions = tf.expand_dims(positions, axis=0)
        positions = tf.tile(positions, [batch_size, 1])
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({"output_dim": self.output_dim, "sequence_length": self.sequence_length})
        return config


In [51]:
input = keras.Input(shape=(19, 19, planes), name='board')
x = layers.Conv2D(50, 1, activation='relu', padding='same',kernel_regularizer=regularizers.l2(0.0001))(input)
x = layers.Dropout(dropout_rate)(x)
x = layers.Conv2D(50, 3, activation='relu', padding='same',kernel_regularizer=regularizers.l2(0.0001))(x)
x1 = MB4D(50)(x)
x=layers.add([x1,x])
x1 = MB4D(50)(x)
x=layers.add([x1,x])
x1 = MB4D(50)(x)
x=layers.add([x1,x])
x = layers.Reshape((-1, 50))(x)
x = PositionalEmbedding(361,50)(x)
x=MB3D(50)(x)
x=MB3D(50)(x)
x = layers.Reshape((19, 19, 50))(x[:, :361, :])

**Define Outputs & Train**

In [53]:
#Define Outputs
policy_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias=False,
                            kernel_regularizer=regularizers.l2(0.0001))(x)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Activation('softmax', name='policy')(policy_head)

value_head = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias=False,
                           kernel_regularizer=regularizers.l2(0.0001))(x)
value_head = layers.Flatten()(value_head)
value_head = layers.Dense(50, activation='relu',
                         kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value',
                         kernel_regularizer=regularizers.l2(0.0001))(value_head)

In [54]:
custom_objects = {"MB4D": MB4D, "MB3D": MB3D, "PositionalEmbedding": PositionalEmbedding}

In [55]:
#Before running any models, you should run custom layers first
def load_model(model_name):
    if os.path.exists(model_name):
        model = keras.models.load_model(model_name, custom_objects=custom_objects)
        return model
    else:
        return None

In [56]:
def model_train_eval(model_name, epochs=epochs):
    model = load_model(model_name)

    if model is None:
        print("Model not found. Creating a new model.")
        model = keras.Model(inputs=input, outputs=[policy_head, value_head])
        model.summary()
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005),
                      loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
                      loss_weights={'policy': 1.0, 'value': 1.0},
                      metrics={'policy': 'categorical_accuracy', 'value': 'mse'})
    else:
        print("Loaded existing model.")
        model.summary()

    for i in range(1, epochs + 1):
        print('epoch ' + str(i))
        golois.getBatch(input_data, policy, value, end, groups, i * N)
        history = model.fit(input_data,
                            {'policy': policy, 'value': value},
                            epochs=1, batch_size=batch)
        if (i % 5 == 0):
            gc.collect()
        if (i % 20 == 0):
            golois.getValidation(input_data, policy, value, end)
            val = model.evaluate(input_data,
                                 [policy, value], verbose=0, batch_size=batch)
            print("val =", val)
            model.save(model_name)

In [ ]:
model_train_eval('HungNGUYEN_cnn.h5')

In [57]:
# Download the saved model file
files.download('HungNGUYEN_cnn.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>